# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bontang,0.13,117.50,84.56,66,12,7.31,ID,1598242083
1,Mount Gambier,-37.83,140.77,51.80,66,90,5.82,AU,1598242128
2,Ushuaia,-54.80,-68.30,32.00,90,40,5.82,AR,1598242120
3,San Jerónimo,-11.95,-75.28,47.19,62,11,3.71,PE,1598242320
4,Guangzhou,23.12,113.25,93.00,55,40,4.47,CN,1598242131
...,...,...,...,...,...,...,...,...,...
569,Grand-Lahou,5.14,-5.02,73.36,90,33,7.61,CI,1598242387
570,San Rafael,-34.62,-68.33,51.94,38,0,2.86,AR,1598242383
571,Assiut,27.18,31.18,75.20,57,0,9.17,EG,1598242388
572,Zhezkazgan,47.80,67.71,66.20,55,75,17.90,KZ,1598242388


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
location = weather_df[["Lat","Lng"]]
humidity = weather_df["Humidity"]

In [4]:
fig = gmaps.figure(center=(46.0,-5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(location, weights = humidity, 
                                 dissipating=False, max_intensity=np.max(humidity),
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_cities = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)]
narrow_cities = narrow_cities.dropna()
narrow_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,Airai,-8.93,125.41,76.66,36,0,7.74,TL,1598242136
24,Miandrivazo,-19.52,45.47,70.14,63,0,2.19,MG,1598242322
109,Abu Samrah,35.30,37.18,71.37,39,0,1.59,SY,1598242332
134,Panjāb,34.39,67.02,70.54,21,0,3.56,AF,1598242336
153,Hūn,29.13,15.95,70.18,67,0,5.44,LY,1598242338
171,Paphos,34.77,32.42,71.60,49,0,8.05,CY,1598242340
240,Zhangjiakou,40.81,114.88,76.37,53,0,6.46,CN,1598242176
246,Diamantino,-14.41,-56.45,72.45,49,0,2.53,BR,1598242134
278,Portland,45.52,-122.68,75.00,63,0,1.01,US,1598242264
334,Baghdad,33.34,44.40,73.40,49,0,2.24,IQ,1598242240


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrow_cities.loc[:,["City", "Country","Lat","Lng"]]

#how to add an empty column to a dataframe
#https://www.kite.com/python/answers/how-to-create-an-empty-column-in-a-pandas-dataframe-in-python
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
7,Airai,TL,-8.93,125.41,
24,Miandrivazo,MG,-19.52,45.47,
109,Abu Samrah,SY,35.30,37.18,
134,Panjāb,AF,34.39,67.02,
153,Hūn,LY,29.13,15.95,


In [7]:
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.append(np.nan)

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
7,Airai,TL,-8.93,125.41,Atsabe
24,Miandrivazo,MG,-19.52,45.47,Miandrivazo
109,Abu Samrah,SY,35.30,37.18,Suruj
134,Panjāb,AF,34.39,67.02,Panjab
153,Hūn,LY,29.13,15.95,Houn
171,Paphos,CY,34.77,32.42,Paphos
240,Zhangjiakou,CN,40.81,114.88,Zhangjiakou
246,Diamantino,BR,-14.41,-56.45,Diamantino
278,Portland,US,45.52,-122.68,Portland
334,Baghdad,IQ,33.34,44.40,Baghdad


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))